In [1]:
#to print all output for a cell instead of only last one 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import and fixed seed 

In [2]:
import os
import sys

import torch
import random
import numpy as np

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

download data (dataset and glove)

In [3]:
#source of this code -> https://gist.github.com/hantoine/c4fc70b32c2d163f604a8dc2a050d5f6 

from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

print("Current work directory: {}".format(os.getcwd()))

data_folder = os.path.join(os.getcwd(),"data")
dataset_folder = os.path.join(data_folder,"dependency_treebank")

if not os.path.exists(data_folder):
    os.makedirs(data_folder)

dataset_url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"

def download_and_unzip(url):
    if not os.path.exists(dataset_folder):
        print("downloading to",dataset_folder)
        with urlopen(url) as response:
            zipfile = ZipFile(BytesIO(response.read()))
            zipfile.extractall(path=data_folder)
    else :
        print("the dataset has been already downloaded")


download_and_unzip(dataset_url)


Current work directory: c:\Users\Riccardo\Project\GitRepo\NLP-assignments\assignment1
the dataset has been already downloaded


In [ ]:
import gensim
import gensim.downloader as gloader

def download_glove_emb():
    if not os.path.exists(data_folder):   #sbagliato 
        
        embedding_dimension=300
        download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)
        emb_model = gloader.load(download_path)
    else :
        print("is it not necessary to download GloVe embeddings at this point")

print(emb_model)

In [29]:
import os
import pandas as pd

#encode dataset in pandas dataframe 

def encode_dataset(dataset_name: str) -> pd.DataFrame:
    dataframe_rows = []

    for doc in os.listdir(dataset_folder):
      doc_num = int(doc[5:8])
      doc_path = os.path.join(dataset_folder,doc)

      with open(doc_path, mode='r', encoding='utf-8') as file:
        df = pd.read_csv(file,sep='\t',header=None,skip_blank_lines=False)
        df.rename(columns={0:'word',1:"TAG",2:"remove"},inplace=True)
        df.drop("remove",axis=1,inplace=True)
        
        df["group_num"] = df.isnull().all(axis=1).cumsum()
        df.dropna(inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        df_list = [df.iloc[rows] for _, rows in df.groupby('group_num').groups.items()]
        for n,d in enumerate(df_list) :
            dataframe_row = {
                "split" : 'train' if doc_num<=100 else ('val' if doc_num<=150  else 'test'),
                "doc_id" : doc_num,
                "sentence_num" : n,
                "words": d['word'].tolist(),
                "tags":  d['TAG'].tolist()
            }
            dataframe_rows.append(dataframe_row)

    dataframe_path = os.path.join(data_folder, dataset_name + ".csv")
    df_final = pd.DataFrame(dataframe_rows)
    df_final.to_csv(dataframe_path)
      
    return  df_final


print("Encoding dataset...")
df = encode_dataset("dataset")
print("Encoding completed!")


Encoding dataset...
Encoding completed!
<bound method NDFrame.head of       split  doc_id  sentence_num  \
0     train       1             0   
1     train       1             1   
2     train       2             0   
3     train       3             0   
4     train       3             1   
...     ...     ...           ...   
3909   test     198             9   
3910   test     198            10   
3911   test     199             0   
3912   test     199             1   
3913   test     199             2   

                                                  words  \
0     [Pierre, Vinken, ,, 61, years, old, ,, will, j...   
1     [Mr., Vinken, is, chairman, of, Elsevier, N.V....   
2     [Rudolph, Agnew, ,, 55, years, old, and, forme...   
3     [A, form, of, asbestos, once, used, to, make, ...   
4     [The, asbestos, fiber, ,, crocidolite, ,, is, ...   
...                                                 ...   
3909  [They, also, said, that, more, than, a, dozen,...   
3910  [Sen., 